# Course project


Спасибо Андрей за курс, не присутствовал на занятиях, но с большим удовольствием несколько раз пересмотрел вебинары. Очень небанальная подача информации, видно, что вы практик и вам есть чем поделиться и вы не плаваете в преджмете (что не всегда случается с преподавателями GB). Вы просили фидбек, плюсы - видна заинтересованность передать свои знания студентам, отвечаете на самые сомнительные вопросы, понравился методология, когда на одном примере мы от курса к курсу улучшали результаты используя все более изощренные модели, а также что каждый вебинар связан с вебинаром до и после - так легче держать весь курс в голове. Из минусов - чуть ли не каждом вебинаре были траблы с запуском кода, постоянные перезапуски на другой операционке. 


Мой курсовой на базе вашего кода. Дополнительно сделал я прямо сказать немного:


1. Написал baseline. Получил 0.185 на тестовой.


2. Решил поблему холодного старта для retail_test1.csv, убрав отуда отсутствующие user_id.


3. Сократил тренировочный датасет, убрав первые 10 недель (дало самый большой прирост метрики).


4. Мои сгенерированные признаки оказались не такими эффективными как ваши, поэтому просто использовал ваш код по генерации признаков.


5. Сгенерировал признак с помощью Word2Vec (улучшение метрики незначительное)


6. Пробовал поработать с кандидатами - генерировать их разными способами, а затем выбирать тех кто пересекался сразу у двух и более моделей, но лучше own_recomend не получил



Метрика составила 0.267





## **Основное**
- Дедлайн - 31 мая 23:59
- На retail_test1.csv целевая метрика precision@5 > 0.235
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. В решении должны быть отчетливо видна метрика на новом тестовом сете из файла retail_test1.csv, то есть вам нужно для всех юзеров из этого файла выдать выши рекомендации, и посчитать на actual покупках precision@5. 

**!! Мы не рассматриваем холодный старт для пользователя, все наши пользователя одинаковы во всех сетах, поэтому нужно позаботиться об их исключении из теста.**


**Hints:** 

Сначала просто попробуйте разные параметры MainRecommender:  
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)  
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)  
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)  
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)  

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

Если вы делаете двухуровневую модель - следите за валидацией 

In [1]:
!pip install implicit==0.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  PATH = '/content/drive/MyDrive/Move_to/GEEK_brain/Рекомендательные системы/webinar_3/'

else:

  PATH = 'D:/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import libs

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

from gensim.models import Word2Vec

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
try:

  from metrics import precision_at_k, recall_at_k
  from utils import prefilter_items
  from recommenders import MainRecommender

except:
  
  sys.path.insert(0,'/content/drive/MyDrive/Move_to/GEEK_brain/Рекомендательные системы/webinar_8/course_project')
  from metrics import precision_at_k, recall_at_k
  from utils import prefilter_items
  from recommenders import MainRecommender

In [4]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")


 # # сырой и простой пример как можно обернуть в функцию
def evalRecall(df_result, target_col_name, recommend_model):
    result_col_name = 'result'
    df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_model(x, N=25))
    return df_result.apply(lambda row: recall_at_k(row[result_col_name], row[ACTUAL_COL], k=N_PREDICT), axis=1).mean()
# evalRecall(result_eval_matcher, USER_COL, recommender.get_own_recommendations)
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()   

## Read data

In [5]:
PATH_DATA = "../../data"

In [6]:
# data = pd.read_csv(os.path.join(PATH_DATA,'retail_train.csv'))
# item_features = pd.read_csv(os.path.join(PATH_DATA,'product.csv'))
# user_features = pd.read_csv(os.path.join(PATH_DATA,'hh_demographic.csv'))



try:
    
  data = pd.read_csv(os.path.join(PATH_DATA,'retail_train.csv'))
  item_features = pd.read_csv(os.path.join(PATH_DATA,'product.csv'))
  user_features = pd.read_csv(os.path.join(PATH_DATA,'hh_demographic.csv'))
  df_test = pd.read_csv('retail_test1.csv')

except:

  data = pd.read_csv(PATH + 'data/retail_train.csv')
  item_features = pd.read_csv(PATH + '/data/product.csv') 
  user_features = pd.read_csv(PATH + '/data/hh_demographic.csv') 
  df_test = pd.read_csv(PATH + '/data/retail_test1.csv')

# Set global const

In [7]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

# N = Neighbors
N_PREDICT = 50 


TOPK_RECALL = 50
TOPK_PRECISION = 5

# Process features dataset

In [8]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

# Baseline

In [9]:
data_baseline = data.copy()

n_items_before = data_baseline['item_id'].nunique()
data_baseline = prefilter_items(data_baseline, take_n_popular=500)
n_items_after = data_baseline['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

recommender_baseline = MainRecommender(data_baseline)

Decreased # items from 89051 to 501


GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/501 [00:00<?, ?it/s]

In [10]:
# ищем общих пользователей
common_users = data_baseline.user_id.values


print_stats_data(df_test,'data_baseline')
df_test = df_test[df_test.user_id.isin(common_users)]
print_stats_data(df_test,'data_baseline')


data_baseline
Shape: (88734, 12) Users: 1885 Items: 20497
data_baseline
Shape: (88724, 12) Users: 1884 Items: 20496


In [11]:
# создаем для baseline
result_baseline = df_test.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_baseline.columns=[USER_COL, ACTUAL_COL]
result_baseline.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [12]:
%%time
result_baseline['own_rec'] = result_baseline[USER_COL].apply(lambda x: recommender_baseline.get_own_recommendations(x, N=N_PREDICT))

CPU times: user 2.23 s, sys: 32.5 ms, total: 2.26 s
Wall time: 2.26 s


In [13]:
sorted(calc_precision(result_baseline, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

[('own_rec', 0.18524416135881103)]

Вывод: получили метрику на тестовом датасете в 0.18, которую и будем улучшать

# Split dataset for train, eval, test

In [14]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)


VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [15]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# убрал слишком 10 cамых старые данные
data_train_matcher = data[data['week_no'] > data['week_no'].min() + 10]


# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]

# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [16]:
# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

In [17]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2304760, 12) Users: 2492 Items: 86513
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [18]:
# выше видим разброс по пользователям и товарам и дальше мы перейдем к warm-start (только известные пользователи)

In [19]:
data_val_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2104867,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,0.0,0.0
2107468,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,0.0,0.0


# Prefilter items

In [20]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/content/drive/MyDrive/Move_to/GEEK_brain/Рекомендательные системы/webinar_8/course_project/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86513 to 5001


# Make cold-start to warm-start

In [21]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]
df_test = df_test[df_test.user_id.isin(common_users)]


print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')
print_stats_data(df_test,'test')

train_matcher
Shape: (942509, 13) Users: 2492 Items: 5001
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329
test
Shape: (88717, 12) Users: 1883 Items: 20496


# Init/train recommender

In [22]:
recommender = MainRecommender(data_train_matcher)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

# Eval recall of matching

In [23]:
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [24]:
%%time
# для понятности расписано все в строчку, без функций, ваша задача уметь оборачивать все это в функции
result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
result_eval_matcher['sim_item_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_items_recommendation(x, N=N_PREDICT))
result_eval_matcher['als_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=N_PREDICT))

CPU times: user 43.5 s, sys: 27.2 s, total: 1min 10s
Wall time: 46.5 s


### Пример оборачивания

### Recall@50 of matching

In [25]:
sorted(calc_recall(result_eval_matcher, TOPK_RECALL), key=lambda x: x[1],reverse=True)

[('own_rec', 0.14215947846084323),
 ('als_rec', 0.07261034542276285),
 ('sim_item_rec', 0.04321735327102476)]

### Precision@5 of matching

In [26]:
sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

[('own_rec', 0.2618384401114206),
 ('als_rec', 0.15246053853296193),
 ('sim_item_rec', 0.0754874651810585)]

# Ranking part

### Обучаем модель 2-ого уровня на выбранных кандидатах

- Обучаем на data_train_ranking
- Обучаем *только* на выбранных кандидатах
- Я *для примера* сгенерирую топ-50 кадидиатов через get_own_recommendations
- (!) Если юзер купил < 50 товаров, то get_own_recommendations дополнит рекоммендации топ-популярными

In [27]:
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 

## Подготовка данных для трейна

In [28]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [29]:
# собираем кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [30]:
df_match_candidates.head(2)

,user_id,candidates
0,2070,"[1105426, 6442443, 879194, 928263, 1092937, 94..."
1,2021,"[950935, 1119454, 835578, 1019142, 863762, 965..."


In [31]:
# разворачиваем товары
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [32]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [33]:
df_match_candidates.head(4)

,user_id,item_id
0,2070,1105426
0,2070,6442443
0,2070,879194
0,2070,928263


### Check warm start

In [34]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (107700, 2) Users: 2154 Items: 4602


### Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1 

In [35]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 
df_ranker_train.head()

,user_id,item_id,target
2104867,2070,1019940,1
2107468,2021,840361,1
2107469,2021,856060,1
2107470,2021,869344,1
2107471,2021,896862,1


In [36]:
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

In [37]:
# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

In [38]:
# дополняем нулями таргет
df_ranker_train['target'].fillna(0, inplace= True)

In [39]:
# проверим дубликаты пар
df_agg = df_ranker_train.groupby(by=['user_id', 'item_id']).count()
df_agg[df_agg.target > 1]

,,target
user_id,item_id,


In [40]:
df_ranker_train.target.value_counts()

0.0    92231
1.0    14887
Name: target, dtype: int64

In [41]:
df_ranker_train.head(2)

,user_id,item_id,target
0,2070,1105426,0.0
1,2070,6442443,0.0


## Подготавливаем фичи для обучения модели

### Описательные фичи

In [42]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [43]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [44]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,6442443,0.0,378,GROCERY,National,BAG SNACKS,POTATO CHIPS,10 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


**Фичи user_id:**
    - Средний чек
    - Средняя сумма покупки 1 товара в каждой категории
    - Кол-во покупок в каждой категории
    - Частотность покупок раз/месяц
    - Долю покупок в выходные
    - Долю покупок утром/днем/вечером

**Фичи item_id**:
    - Кол-во покупок в неделю
    - Среднее ол-во покупок 1 товара в категории в неделю
    - (Кол-во покупок в неделю) / (Среднее ол-во покупок 1 товара в категории в неделю)
    - Цена (Можно посчитать из retil_train.csv)
    - Цена / Средняя цена товара в категории
    
**Фичи пары user_id - item_id**
    - (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)
    - (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
    - (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)

### Поведенческие фичи

##### Чтобы считать поведенческие фичи, нужно учесть все данные что были до data_val_ranker

In [45]:
df_join_train_matcher.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
89187,393,27870814069,76,834117,1,3.79,427,-1.2,113,12,0.0,0.0
89848,1770,27870963541,76,871999,1,4.49,432,0.0,44,12,0.0,0.0
89849,1770,27870963541,76,1109939,1,3.69,432,0.0,44,12,0.0,0.0
89850,1770,27870963541,76,8019371,1,13.99,432,0.0,44,12,0.0,0.0
89851,1770,27870963541,76,8068502,1,1.99,432,0.0,44,12,0.0,0.0


## !!! Пока выполните нотбук без этих строк, потом вернитесь и запустите их, обучите ранкер и посмотрите на метрики с ранжированием

In [46]:
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().rename('total_item_sales_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('total_quantity_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().rename('total_user_sales_value'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_baskter')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)


## Парная фича

In [47]:
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=[USER_COL,ITEM_COL]).agg('quantity').sum().rename('user_item_total_quantity'), how='left',on=[USER_COL,ITEM_COL])

In [48]:
df_ranker_train.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,item_freq,user_freq,total_user_sales_value,item_quantity_per_week,user_quantity_per_week,item_quantity_per_basket,user_quantity_per_baskter,item_freq_per_basket,user_freq_per_basket,user_item_total_quantity
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,...,97,2619,7599.15,1.357143,1658.511905,0.000461,0.563533,0.000392,0.010594,7.0
1,2070,6442443,0.0,378,GROCERY,National,BAG SNACKS,POTATO CHIPS,10 OZ,45-54,...,55,2619,7599.15,0.654762,1658.511905,0.000222,0.563533,0.000222,0.010594,NaN
2,2070,879194,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,45-54,...,45,2619,7599.15,0.630952,1658.511905,0.000214,0.563533,0.000182,0.010594,3.0
3,2070,928263,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,13 CT,45-54,...,64,2619,7599.15,0.845238,1658.511905,0.000287,0.563533,0.000259,0.010594,3.0
4,2070,1092937,1.0,1089,MEAT-PCKGD,National,LUNCHMEAT,BOLOGNA,16OZ,45-54,...,455,2619,7599.15,6.964286,1658.511905,0.002366,0.563533,0.001840,0.010594,11.0


## Получим эмбединги товаров c помощью Word2Vec

In [49]:
# Функция для получения среднего ембэдинга покупок пользователя
 
def word2vec_mean(user_id):
    try:
        return np.mean([w2v_model.wv[str(itemid)] for itemid in df_[df_['user_id'] == user_id].item_id.values[0]])
    except:
        return 0

In [50]:
# сформируем пару item-users из data_train_ranker
df_ = data_train_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()

# Представим для удобства данные в виде списка  
my_p = [[str(item) for item in df_[df_['user_id'] == user].item_id.values[0]] for user in df_['user_id']]

# инициализируем и обучаем модель
w2v_model = Word2Vec(min_count=1, sg=1, workers=3)
w2v_model.build_vocab(my_p, progress_per=100)
w2v_model.train(my_p, total_examples=w2v_model.corpus_count, epochs=10, report_delay=1)

(1413928, 1419410)

In [51]:
# Генерируем фичу
df_ranker_train['mean_embedofitem_user'] = df_ranker_train[USER_COL].apply(lambda x: word2vec_mean(x))

In [52]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [53]:
X_train.columns

Index(['user_id', 'item_id', 'manufacturer', 'department', 'brand',
       'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product',
       'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc',
       'hh_comp_desc', 'household_size_desc', 'kid_category_desc',
       'total_item_sales_value', 'total_quantity_value', 'item_freq',
       'user_freq', 'total_user_sales_value', 'item_quantity_per_week',
       'user_quantity_per_week', 'item_quantity_per_basket',
       'user_quantity_per_baskter', 'item_freq_per_basket',
       'user_freq_per_basket', 'user_item_total_quantity',
       'mean_embedofitem_user'],
      dtype='object')

In [54]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')


## Обучение модели ранжирования

In [55]:
%%time
lgb = LGBMClassifier(objective='binary',
                     max_depth=10,
                     n_estimators=200,
                     learning_rate=0.1,
                     categorical_column=cat_feats,
                     n_jobs=100)

lgb.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


CPU times: user 46 s, sys: 14.9 s, total: 1min
Wall time: 37 s


In [56]:
train_preds = lgb.predict_proba(X_train)

In [57]:
df_ranker_predict = df_ranker_train.copy()

In [58]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

## Подведем итоги

    Мы обучили модель ранжирования на покупках из сета data_train_ranker и на кандитатах от own_recommendations, что является тренировочным сетом, и теперь наша задача предсказать и оценить именно на тестовом сете.

# Evaluation on test dataset

In [59]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


## Eval matching on test dataset

In [60]:
%%time
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

CPU times: user 5.39 s, sys: 44.8 ms, total: 5.43 s
Wall time: 5.41 s


In [61]:
# померяем precision только модели матчинга, чтобы понимать влияение ранжирования на метрики

sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

[('own_rec', 0.21645445641527913)]

## Eval re-ranked matched result on test dataset
    Вспомним df_match_candidates сет, который был получен own_recommendations на юзерах, набор пользователей мы фиксировали и он одинаков, значи и прогноз одинаков, поэтому мы можем использовать этот датафрейм для переранжирования.
    

In [62]:
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [63]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

## Проверьте данные метрики с фичами и без (PS: должен быть прирост)

In [64]:
# смотрим на метрики выше и сравниваем что с ранжированием и без, добавляем фичи и то же смотрим
# в первом приближении метрики должны расти с использованием второго этапа

print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.2679540709812108)
('own_rec', 0.21645445641527913)


/content/drive/MyDrive/Move_to/GEEK_brain/Рекомендательные системы/webinar_8/course_project/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


# Оценка на тесте для выполнения курсового проекта

In [65]:
df_test.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0
4,1602,41665647035,664,927712,1,0.59,334,-0.4,1741,96,0.0,0.0


In [66]:
result_test = df_test.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_test.columns=[USER_COL, ACTUAL_COL]
result_test.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


Берем топ-k предсказаний, ранжированных по вероятности, для каждого юзера

# Считаем precision@5 по новому тесту

In [67]:
result_test['my_prediction'] = result_test[USER_COL].apply(lambda user_id: rerank(user_id))

In [68]:
print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.2679540709812108)
('own_rec', 0.21645445641527913)


/content/drive/MyDrive/Move_to/GEEK_brain/Рекомендательные системы/webinar_8/course_project/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


# Помощник для решения курсового

Постановка задачи + особенности данных + подход к решению из [презентации](https://vk.com/away.php?utf=1&to=https%3A%2F%2Fgithub.com%2Faprotopopov%2Fretailhero_recommender%2Fblob%2Fmaster%2Fslides%2Fretailhero_recommender.pdf) решения 1-ого места

Бонус - [код](https://github.com/aprotopopov/retailhero_recommender) решения 1-ого места

### Генерация кандидатов

В основном генерировали k = {50, 100} кандидатов, чтобы модель 2-ого уровня отрабатывала за разумное время

- Прошлые покупки юзера + топ-популярных, если покупок < k --> recall@100 ~ 42%
- Из ALS, item2item моделей
- implicit.nn.CosineRecommender

### Фичи в модели 2-ого уровня
Коллаборативные:
- biases + embeddings из коллаборативной фильтрации / Скоры item2item моделей
- TF-IDF матирца item-ов c > N покупками
- TF-IDF + TSNE/UMAP

handcrafted фичи для товаров:
- категории товаров
- нормированная частота покупки товара для каждого клиента
- кол-во магазинов, в которых продавался товар
- кол-во транзакций клиента
- mean / max / std кол-ва уникальных товаров в корзине клиента 
- mean / max / std кол-ва уникальных категорий в корзине клиента 

handcrafted фичи для юзеров:
- Средний чек
- Средняя цена одного купленного товара
- Среднее кол-во дней между покупками / с последней покупки    
- Количество уникальных покупок по всем категориям transaction_id, product_id, store_id, level_i_id
- Признаки с накопленными бонусами
- Средняя скидка, доля купленных товаров со скидками

Интересные:
- Факт заказа каждого товара в последних 5 транзакциях в виде последовательности бит (категориальная фича). 
10001 - купил товар в последней транзакции и 5 транзакий назад (feature hashing)
- item co_ocurrence
- word2vec эмбеддинги товаров (альтернативное название - item2vec, prod2vec)
- Расстояние от word2vec эмбеддинга товара до среднего эмбедиингов товаров, купленных юзером

### Модели 2-ого уровня
- классификация через LightGBM
- ранжирование CatBoost через YetiRank, YetiRankPairwise